### スコアリングフェーズにおける前処理

### モデルデータの読み込み

In [1]:
import pandas as pd
# csvからデータを読み込み
df = pd.read_csv('./data/av_loan_u6lujuX_CVtuZ9i.csv', header=0)
# 最終列を正解データとしてdfに格納する.
X = df.iloc[:,:-1]
ID = X.iloc[:,[0]]
X = X.drop('Loan_ID', axis=1)
y = df.iloc[:,-1]

print('---------------------')
print('Raw shape:', df.shape)
print(df.dtypes)
# スコアで型を一致させる為保持
df_dtypes = df.dtypes
print('X shape:', X.shape)
display(df.head())

---------------------
Raw shape: (614, 13)
Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object
X shape: (614, 11)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


### モデル用データの前処理

In [2]:
# mapを使用しclass_mappingのdictを適応する。
class_mapping = {'N':1, 'Y':0}
y = y.map(class_mapping)
print('---------------------')
print(y.value_counts())
print('---------------------')
print(ID.join(X).join(y).dtypes)
display(ID.join(X).join(y).head())

# 表示列数のオプション変更
pd.options.display.max_columns = 50

---------------------
0    422
1    192
Name: Loan_Status, dtype: int64
---------------------
Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status            int64
dtype: object


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,1
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,0


### モデル用データの前処理（One-hotエンコーディング）

In [3]:
# One-hotエンコーディングを行う変数を指定する。
ohe_columns = ['Dependents', 'Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']

X_ohe = pd.get_dummies(X, dummy_na=True, columns=ohe_columns, drop_first=True)

print('X_ohe shape:', X_ohe.shape)
display(X_ohe.head())

X_ohe shape: (614, 20)


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Dependents_2,Dependents_3+,Dependents_nan,Gender_Male,Gender_nan,Married_Yes,Married_nan,Education_Not Graduate,Education_nan,Self_Employed_Yes,Self_Employed_nan,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan
0,5849,0.0,NaN,360.0,1.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,4583,1508.0,128.0,360.0,1.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0
2,3000,0.0,66.0,360.0,1.0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0
3,2583,2358.0,120.0,360.0,1.0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0
4,6000,0.0,141.0,360.0,1.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


### モデル用データの前処理（欠損値補完）

In [4]:
from sklearn.impute import SimpleImputer

# 欠損値NaNを平均値（mean）で置換
imp = SimpleImputer()
imp.fit(X_ohe)

# 学習済みImputerをtransformで適応する
X_ohe_columns = X_ohe.columns.values
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)

# 結果表示
display(X_ohe.head())

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Dependents_2,Dependents_3+,Dependents_nan,Gender_Male,Gender_nan,Married_Yes,Married_nan,Education_Not Graduate,Education_nan,Self_Employed_Yes,Self_Employed_nan,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan
0,5849.0,0.0,146.412162,360.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4583.0,1508.0,128.000000,360.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3000.0,0.0,66.000000,360.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,2583.0,2358.0,120.000000,360.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,6000.0,0.0,141.000000,360.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### モデル用データの前処理（次元圧縮）

In [5]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

selector = RFE(RandomForestClassifier(n_estimators=100, random_state=1),
               n_features_to_select=10,
               step=.05)

selector.fit(X_ohe, y)

X_fin = pd.DataFrame(selector.transform(X_ohe),
                     columns=X_ohe_columns[selector.support_])

print('X_fin shape:', X_fin.shape)
display(X_fin.head())

X_fin shape: (614, 10)


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Gender_Male,Married_Yes,Education_Not Graduate,Property_Area_Semiurban
0,5849.0,0.0,146.412162,360.0,1.0,0.0,1.0,0.0,0.0,0.0
1,4583.0,1508.0,128.000000,360.0,1.0,1.0,1.0,1.0,0.0,0.0
2,3000.0,0.0,66.000000,360.0,1.0,0.0,1.0,1.0,0.0,0.0
3,2583.0,2358.0,120.000000,360.0,1.0,0.0,1.0,1.0,1.0,0.0
4,6000.0,0.0,141.000000,360.0,1.0,0.0,1.0,0.0,0.0,0.0


### スコア用データの前処理
- モデルデータと同じカラム名、同じtypeに変換する

In [6]:
# モデル用データとデータ型が異なるためカラム選択はobjectで読み込み為の準備を行う
print(f'モデルデータのデータ型\n{df_dtypes}')
# カラムにobjectを指定する方法 
# my_dtype = {}
# for column in ohe_columns:
#     my_dtype[column] = object
# print(my_dtype)
# モデル用データのdtypesをそのまま当てはめる方法
# dict(df_dtypes)

# dtypeにdictで指定できる
df_s = pd.read_csv('./data/av_loan_test_Y3wMUE5_7gLdaTN.csv', header=0, dtype=dict(df_dtypes))
ID_s = df_s.iloc[:,[0]]
X_s = df_s.drop('Loan_ID', axis=1)

print(f'テストデータのデータ型\n{X_s.dtypes}')
print('Raw shape:', df_s.shape)
print('X shape:',X_s.shape)
print('-----------------------')

display(df_s.head())

モデルデータのデータ型
Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object
テストデータのデータ型
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
dtype: object
Raw shape: (333, 12)
X shape: (333, 11)
-----------------------


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0.0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500.0,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800.0,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546.0,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0.0,78.0,360.0,1.0,Urban


### スコア用データの前処理（One-hotエンコーディング）
- モデルデータとは独立に行う

In [7]:
X_ohe_s = pd.get_dummies(X_s, dummy_na=True, columns=ohe_columns, drop_first=True)

print('X_ohe_s shape:', X_ohe_s.shape)
display(X_ohe_s.head())

X_ohe_s shape: (333, 20)


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Dependents_2,Dependents_nan,Gender_Male,Gender_Unknown,Gender_nan,Married_Yes,Married_nan,Education_Not Graduate,Education_nan,Self_Employed_Yes,Self_Employed_nan,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan
0,5720,0.0,110.0,360.0,1.0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
1,3076,1500.0,126.0,360.0,1.0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0
2,5000,1800.0,208.0,360.0,1.0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0
3,2340,2546.0,100.0,360.0,NaN,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0
4,3276,0.0,78.0,360.0,1.0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0


### スコア用データの前処理（One-hotエンコーディングの整合チェック）

In [8]:
# Pythonの集合型変数を利用する
cols_model = set(X_ohe.columns.values)
cols_score = set(X_ohe_s.columns.values)

# モデルにはあってスコアには無いデータ項目
diff1 = cols_model - cols_score
print('Modelのみ:', diff1)

# スコアには会ってモデルには無いデータ項目
diff2 = cols_score - cols_model
print('Scoreのみ:',diff2)

Modelのみ: {'Dependents_3+'}
Scoreのみ: {'Gender_Unknown'}


### スコア用データの前処理（One-hotエンコーディング後の不整合解消）

In [9]:
# カラム名のみのdfを作成する
df_cols_m = pd.DataFrame(None, columns=X_ohe_columns, dtype=float)
display(df_cols_m)

# 上記カラムにスコア用データをconcatする、該当しないカラムは欠損状態で結合される
X_ohe_s2 = pd.concat([df_cols_m, X_ohe_s], sort=False)
print(X_ohe_s2.shape)
display(X_ohe_s2.head())

# スコアリングに登場したGender_Unknownを削除する
X_ohe_s3 = X_ohe_s2.drop(list(diff2), axis=1)
print(X_ohe_s3.shape)
display(X_ohe_s3.head())

# モデルに登場したDepwndents_3+のNaNをゼロ埋めする
X_ohe_s3.loc[:,list(diff1)] = X_ohe_s3.loc[:,list(diff1)].fillna(0, axis=1)
display(X_ohe_s3.head())

# モデルでfitさせたRFEをスコアで使用する為に、順序をモデルと統一する
X_ohe_s3 = X_ohe_s3.reindex(X_ohe.columns.values, axis=1)
display(X_ohe_s3.head())

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Dependents_2,Dependents_3+,Dependents_nan,Gender_Male,Gender_nan,Married_Yes,Married_nan,Education_Not Graduate,Education_nan,Self_Employed_Yes,Self_Employed_nan,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan


(333, 21)


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Dependents_2,Dependents_3+,Dependents_nan,Gender_Male,Gender_nan,Married_Yes,Married_nan,Education_Not Graduate,Education_nan,Self_Employed_Yes,Self_Employed_nan,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan,Gender_Unknown
0,5720.0,0.0,110.0,360.0,1.0,0.0,0.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,3076.0,1500.0,126.0,360.0,1.0,1.0,0.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,5000.0,1800.0,208.0,360.0,1.0,0.0,1.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2340.0,2546.0,100.0,360.0,NaN,0.0,1.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,3276.0,0.0,78.0,360.0,1.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


(333, 20)


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Dependents_2,Dependents_3+,Dependents_nan,Gender_Male,Gender_nan,Married_Yes,Married_nan,Education_Not Graduate,Education_nan,Self_Employed_Yes,Self_Employed_nan,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan
0,5720.0,0.0,110.0,360.0,1.0,0.0,0.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3076.0,1500.0,126.0,360.0,1.0,1.0,0.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5000.0,1800.0,208.0,360.0,1.0,0.0,1.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2340.0,2546.0,100.0,360.0,NaN,0.0,1.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3276.0,0.0,78.0,360.0,1.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Dependents_2,Dependents_3+,Dependents_nan,Gender_Male,Gender_nan,Married_Yes,Married_nan,Education_Not Graduate,Education_nan,Self_Employed_Yes,Self_Employed_nan,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan
0,5720.0,0.0,110.0,360.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3076.0,1500.0,126.0,360.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5000.0,1800.0,208.0,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2340.0,2546.0,100.0,360.0,NaN,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3276.0,0.0,78.0,360.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Dependents_2,Dependents_3+,Dependents_nan,Gender_Male,Gender_nan,Married_Yes,Married_nan,Education_Not Graduate,Education_nan,Self_Employed_Yes,Self_Employed_nan,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan
0,5720.0,0.0,110.0,360.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3076.0,1500.0,126.0,360.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5000.0,1800.0,208.0,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2340.0,2546.0,100.0,360.0,NaN,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3276.0,0.0,78.0,360.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


### スコア用データの前処理（数値変数の欠損値対応）

In [12]:
print('欠損個数（数値変数の欠損補完前）', X_ohe_s3.isnull().sum().sum())

# モデルでfitしたimpでtransformを行う
X_ohe_s4 = pd.DataFrame(imp.transform(X_ohe_s3), columns=X_ohe_columns)

print('欠損個数（数値変数の欠損補完後）', X_ohe_s4.isnull().sum().sum())

欠損個数（数値変数の欠損補完前） 34
欠損個数（数値変数の欠損補完後） 0


### スコア用データの前処理（特徴量選択）

In [16]:
# RFEでfitしたモデルデータの選択された特徴量をスコアに適応する
X_fin_s = X_ohe_s4.loc[:, X_ohe_columns[selector.support_]]
print(X_fin_s.shape)
display(X_fin_s.head())

(333, 10)


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Gender_Male,Married_Yes,Education_Not Graduate,Property_Area_Semiurban
0,5720.0,0.0,110.0,360.0,1.000000,0.0,1.0,1.0,0.0,0.0
1,3076.0,1500.0,126.0,360.0,1.000000,1.0,1.0,1.0,0.0,0.0
2,5000.0,1800.0,208.0,360.0,1.000000,0.0,1.0,1.0,0.0,0.0
3,2340.0,2546.0,100.0,360.0,0.842199,0.0,1.0,1.0,0.0,0.0
4,3276.0,0.0,78.0,360.0,1.000000,0.0,1.0,0.0,1.0,0.0
